<a href="https://colab.research.google.com/github/Brennan-Richards/Research-Recommendation/blob/main/Academic_Research_Recommendation_Dataset_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Prelim



In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 79 kB 2.1 MB/s 
     |████████████████████████████████| 4.0 MB 13.9 MB/s 
     |████████████████████████████████| 1.2 MB 29.4 MB/s 
     |████████████████████████████████| 77 kB 3.6 MB/s 
     |████████████████████████████████| 596 kB 43.3 MB/s 
     |████████████████████████████████| 6.6 MB 31.8 MB/s 
     |████████████████████████████████| 880 kB 21.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=148b575ed1b15cc550c002675fdd01bd5a0c55fa81c5d10965462b4f40678084
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=d9743feb611db11152077668f05b1ba06ab1c1f0e146d6e0b69843d6550c54c1
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sentence-transformers sa

In [ ]:
import pandas as pd
import os
import math
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir('/')
os.chdir('content/drive/Shareddrives/CSC561 Finals')

In [ ]:
!ls

'Choosing Final Project Framework.gdoc'   tags_by_user.csv
'CiteULike Dataset.gdoc'		  tags.dat
'Final Project Dataset Engineering'	  tags_df_w_embed_dist.csv
'Fine Tuning GPT-2 HuggingFace'		  users.dat
'FT^2 GPT2 Model.ipynb'


In [ ]:
tags_df = pd.read_csv('tags.dat', header=None)
tags_df.head(3)

,0
0,ucsc-browser
1,rr_interval
2,userscripts


In [ ]:
tags_df.describe()

,0
count,46389
unique,46389
top,ucsc-browser
freq,1


In [ ]:
users_df = pd.read_csv('users.dat', header=None)
users_df.head(3)

,0
0,70 495 1631 2317 2526 2846 2931 3171 3297 3332...
1,38 493 942 1519 1843 1844 1896 2819 3391 5031 ...
2,20 517 791 800 1329 1767 1984 2126 3009 4458 4...


In [ ]:
users_df.describe()

,0
count,5551
unique,5411
top,137 1791 3631 4130 6290 9954 10539 11100 12830...
freq,38


### Data cleaning

In [ ]:
!pip install wordninja

     |████████████████████████████████| 541 kB 5.3 MB/s 
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541551 sha256=88805e3530f1cc1c9449fcb5feee578099695ef7d9e202356dc499abb66a4eef
  Stored in directory: /root/.cache/pip/wheels/dd/3f/eb/a2692e3d2b9deb1487b09ba4967dd6920bd5032bfd9ff7acfc
Successfully built wordninja


In [ ]:
# Comes from: https://stackoverflow.com/questions/8870261/how-to-split-text-without-spaces-into-list-of-words
import wordninja
wordninja.split('machinelearning')

import nltk
from nltk.corpus import stopwords, words
nltk.download('stopwords')
nltk.download('words')

import itertools

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

#### Tags

In [ ]:
tags_df.dtypes

0    object
dtype: object

In [ ]:
"laplacian" in words.words()

False

In [ ]:
stop_words = set(stopwords.words('english'))
stop_words.add("nan")


In [ ]:
"nan" in stop_words

True

In [ ]:
tags_df.columns

Int64Index([0], dtype='int64')

In [ ]:
tags_df[0] = tags_df[0].apply(lambda x: str(x))

counter = 0

stop_words = set(stopwords.words('english'))
stop_words.add("nan")
stop_words.add("ict")

def clean_tag(raw_tag, alpha=True, remove_stop_words=False):
  """
  Clean an individual tag.
  In: raw tag values
  Out: cleaned tags
  """
  global stop_words
  # Remove '-' and '_' and replace each instance with a space
  raw_tag = raw_tag.replace('-', ' ').replace('_', ' ')

  # Create a list from the tag in order to operate on each token separately.
  raw_tag_word_list = raw_tag.split(' ') if ' ' in raw_tag else [raw_tag]

  # If there are spaces in the tag, don't try to split it further.
  if (not ' ' in raw_tag):
    raw_tag_word_list = wordninja.split(raw_tag) # If no spaces, try to split into words.

  # Remove tokens that are not alphabetic
  if alpha:
    raw_tag_word_list = [word for word in raw_tag_word_list if word.isalpha()]

  # Filter out stop words
  if remove_stop_words:
    raw_tag_word_list = [w for w in raw_tag_word_list if not w in stop_words]

  # Remove SHORT tokens IF NOT words
  clean_word_list = []
  for w in raw_tag_word_list:
    if len(w) <= 3: # If longer than three, just keep it
      if not (w in words.words()): # If the short token is not a word
        clean_word_list.append(None)
        break
    clean_word_list.append(w)

  global counter
  counter += 1

  clean_word_list = [clean_tag for clean_tag in clean_word_list if clean_tag]

  clean_string = " ".join( clean_word_list )
  if counter % 1000 == 0:
    print("COUNTER: ", counter)
    print(clean_string)

  return

In [ ]:
tags_df['clean'] = tags_df[0].apply( lambda dirty_tag: clean_tag(dirty_tag) )

COUNTER:  1000
COUNTER:  2000
COUNTER:  3000
COUNTER:  4000
COUNTER:  5000
COUNTER:  6000
COUNTER:  7000
COUNTER:  8000
COUNTER:  9000
COUNTER:  10000
COUNTER:  11000
COUNTER:  12000
COUNTER:  13000
COUNTER:  14000
COUNTER:  15000
COUNTER:  16000
COUNTER:  17000
COUNTER:  18000
COUNTER:  19000
COUNTER:  20000
COUNTER:  21000
COUNTER:  22000
COUNTER:  23000
COUNTER:  24000
COUNTER:  25000
COUNTER:  26000
COUNTER:  27000
COUNTER:  28000
COUNTER:  29000
COUNTER:  30000
COUNTER:  31000
COUNTER:  32000
COUNTER:  33000
COUNTER:  34000
COUNTER:  35000
COUNTER:  36000
COUNTER:  37000
COUNTER:  38000
COUNTER:  39000
COUNTER:  40000
COUNTER:  41000
COUNTER:  42000
COUNTER:  43000
COUNTER:  44000
COUNTER:  45000
COUNTER:  46000


In [ ]:
# tags_df[tags_df[0].str.contains('nan')]
tags_df[tags_df['clean'] == 'nan']


,0,clean


In [ ]:
tags_df.iloc[4538], tags_df.iloc[39734] = '', ''
tags_df[tags_df['clean'] == 'nan']

,0,clean


In [ ]:
# tags_df[tags_df[0].str.contains('ict')]
tags_df[tags_df['clean'] == 'ict']

,0,clean


In [ ]:
tags_df.to_csv('clean_tags.csv', index=False)

In [ ]:
tags_df.head(11)

,0,clean
0,ucsc-browser,ucsc browser
1,rr_interval,
2,userscripts,user scripts
3,newsciencenetwork,new science network
4,sequence_similarity_search,sequence similarity search
5,naturalcapital,natural capital
6,spiders,spiders
7,localized,localized
8,segmentation_algorithm,segmentation algorithm
9,polymer_physics,polymer physics


In [ ]:
tags_df.describe()

,0,clean
count,46391,46391
unique,46390,33911
top,nan,
freq,2,5368


In [ ]:
# Embed the clean tags and compute a score between
# each tag and "machine learning"
import scipy
from scipy.spatial import distance
# a = (1, 2, 3)
# b = (4, 5, 6)

# print(distance.euclidean(a, b))

In [ ]:
embeddings = model.encode(tags_df['clean']) # Up to 1,000 took 4 seconds
embeddings[0].shape

(384,)

In [ ]:
embeddings[0]

array([-1.54720917e-01, -2.06067473e-01,  2.16859058e-01, -3.46472323e-01,
       -1.17358305e-01,  1.07612852e-02, -1.46372533e+00,  1.08464599e-01,
        3.68896157e-01,  5.38147092e-01, -3.85670483e-01,  9.23044086e-02,
        5.94726443e-01, -1.31570116e-01, -1.52313322e-01, -3.74122888e-01,
        1.61289126e-01,  3.75140598e-03,  5.55340767e-01,  2.95757115e-01,
       -3.88226807e-01, -3.26888084e-01, -2.02434570e-01, -1.79541521e-02,
        2.04652697e-01, -2.54718155e-01, -1.35419488e-01, -4.23124015e-01,
       -2.41202325e-01, -6.16965890e-01,  5.30160889e-02, -3.63600641e-01,
        5.17265797e-01, -2.94020437e-02, -4.45430912e-02, -2.35092282e-01,
        4.57142115e-01, -4.24833238e-01, -3.91976744e-01, -4.32005972e-01,
       -5.16097486e-01,  1.76149711e-01, -4.90433350e-02,  4.63608801e-01,
       -7.41916150e-02, -5.79049468e-01, -1.88058242e-01, -2.92755783e-01,
        1.55055389e-01,  6.56577528e-01, -1.64326847e-01,  2.43120641e-01,
       -5.77566206e-01, -

In [ ]:
embedding_anchors = ['machine learning', 'mathematics', 'engineering', 'science', 'technology', 'computer science', 'neuroscience']
anchor_idxs = []

for a in embedding_anchors:
  idx = tags_df[tags_df['clean'] == a].index[0]
  anchor_idxs.append(idx)

anchor_idxs

[12946, 15678, 19332, 4380, 1151, 1971, 41201]

In [ ]:
tags_df[tags_df['clean'] == 'computer science']

,0,clean
1971,computer-science,computer science
8636,computer_science_vs_mathematics,computer science
28726,0_computer_science,computer science
36095,computer_science,computer science
45124,computerscience,computer science


In [ ]:
print(len(tags_df[tags_df['clean'] == '']), "empty values in clean column")

5370 empty values in clean column


In [ ]:
assert distance.euclidean(embeddings[28726], embeddings[36095]) == 0.0
assert distance.euclidean(embeddings[12946], embeddings[0]) != 0.0

In [ ]:
# Add a column to the tags_df which represents the distance between tag in this row and each 'anchor' tag

for anchor_name, anchor_idx in zip(embedding_anchors, anchor_idxs):
  print(anchor_name, anchor_idx)
  tags_df[f"{ '_'.join( anchor_name.split(' ') ) }_embed_dist"] = [distance.euclidean(embeddings[anchor_idx], embeddings[i]) for i in range( len(tags_df['clean']) ) ]

machine learning 12946
mathematics 15678
engineering 19332
science 4380
technology 1151
computer science 1971
neuroscience 41201


In [ ]:
# tags_df[tags_df['clean'].contains('nan')]

In [ ]:
tags_df.head(10)

,0,clean,machine_learning_embed_dist,mathematics_embed_dist,engineering_embed_dist,science_embed_dist,technology_embed_dist,computer_science_embed_dist,neuroscience_embed_dist
0,ucsc-browser,ucsc browser,10.580732,11.569860,10.340052,10.595269,9.772329,9.110353,10.585641
1,rr_interval,,8.557783,8.242620,8.157562,7.490930,7.857063,8.760796,8.340920
2,userscripts,user scripts,10.383705,11.024372,10.061990,10.910008,10.054292,10.206070,10.729193
3,newsciencenetwork,new science network,8.051067,9.715636,9.094267,6.698873,7.664272,7.268487,8.958341
4,sequence_similarity_search,sequence similarity search,8.924513,10.000997,10.749337,9.764569,10.834929,9.696523,10.809768
5,naturalcapital,natural capital,11.122601,11.198635,10.671456,9.772196,10.377213,10.500147,11.501794
6,spiders,spiders,11.348691,11.548071,11.018040,10.392562,11.304908,11.470849,10.778779
7,localized,localized,10.511032,10.053268,10.188346,9.255118,10.147845,10.172532,10.273067
8,segmentation_algorithm,segmentation algorithm,9.338987,9.845531,10.218202,10.681505,10.218020,10.441625,10.842655
9,polymer_physics,polymer physics,10.643703,9.851032,9.170920,8.711931,9.719798,9.331155,10.924102


In [ ]:
# Export the tags dataset as a CSV
tags_df.to_csv('tags_df_w_embed_dist.csv', index=False)

In [ ]:
# NOTE: Don't necessarily want to drop duplicates because each row (index)
# should map to a value.
# df = df.drop_duplicates('clean')
# df.describe()

#### Users

In [ ]:
tags_df = pd.read_csv('tags_df_w_embed_dist.csv')
tags_df.head()

,0,clean,machine_learning_embed_dist,mathematics_embed_dist,engineering_embed_dist,science_embed_dist,technology_embed_dist,computer_science_embed_dist,neuroscience_embed_dist
0,ucsc-browser,ucsc browser,10.580732,11.569860,10.340052,10.595269,9.772329,9.110353,10.585641
1,rr_interval,NaN,8.557783,8.242620,8.157562,7.490930,7.857063,8.760796,8.340920
2,userscripts,user scripts,10.383705,11.024372,10.061990,10.910008,10.054292,10.206070,10.729193
3,newsciencenetwork,new science network,8.051067,9.715636,9.094267,6.698873,7.664272,7.268487,8.958341
4,sequence_similarity_search,sequence similarity search,8.924513,10.000997,10.749337,9.764569,10.834929,9.696523,10.809768


In [ ]:
tags_df['clean'] = tags_df['clean'].apply(lambda x: str(x) if str(x) != 'nan' else '')
tags_df.head()

,0,clean,machine_learning_embed_dist,mathematics_embed_dist,engineering_embed_dist,science_embed_dist,technology_embed_dist,computer_science_embed_dist,neuroscience_embed_dist
0,ucsc-browser,ucsc browser,10.580732,11.569860,10.340052,10.595269,9.772329,9.110353,10.585641
1,rr_interval,,8.557783,8.242620,8.157562,7.490930,7.857063,8.760796,8.340920
2,userscripts,user scripts,10.383705,11.024372,10.061990,10.910008,10.054292,10.206070,10.729193
3,newsciencenetwork,new science network,8.051067,9.715636,9.094267,6.698873,7.664272,7.268487,8.958341
4,sequence_similarity_search,sequence similarity search,8.924513,10.000997,10.749337,9.764569,10.834929,9.696523,10.809768


In [ ]:
str(tags_df['clean'].iloc[1])

''

In [ ]:
tags_df[tags_df['clean'] == 'nan']

,0,clean,machine_learning_embed_dist,mathematics_embed_dist,engineering_embed_dist,science_embed_dist,technology_embed_dist,computer_science_embed_dist,neuroscience_embed_dist


In [ ]:
tags_df.describe()

,machine_learning_embed_dist,mathematics_embed_dist,engineering_embed_dist,science_embed_dist,technology_embed_dist,computer_science_embed_dist,neuroscience_embed_dist
count,46391.000000,46391.000000,46391.000000,46391.000000,46391.000000,46391.000000,46391.000000
mean,9.763035,9.944162,9.747988,9.357438,9.602904,9.767419,9.956846
std,0.884885,0.992953,0.937067,1.046557,1.017335,0.848706,0.962043
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.115951,9.337698,9.153152,8.836790,9.002088,9.114495,9.435881
50%,9.835248,10.067299,9.867455,9.538224,9.768163,9.831660,10.070407
75%,10.398093,10.666926,10.436957,10.087656,10.344716,10.381740,10.616707
max,12.635427,12.903815,13.066718,12.360171,12.419037,12.588457,13.071475


In [ ]:
tags_df.describe().index

Index(['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'], dtype='object')

In [ ]:
users_df.head()

,0
0,70 495 1631 2317 2526 2846 2931 3171 3297 3332...
1,38 493 942 1519 1843 1844 1896 2819 3391 5031 ...
2,20 517 791 800 1329 1767 1984 2126 3009 4458 4...
3,12 706 709 721 755 756 776 892 895 2588 2991 4...
4,12 761 2678 4034 4035 5439 5494 5758 7798 9046...


In [ ]:
users_df.describe()

,0
count,5551
unique,5411
top,137 1791 3631 4130 6290 9954 10539 11100 12830...
freq,38


In [ ]:

user_counter = 0

def test_similarity(tag_num, similarity_threshold):
  """
  Check if the given tag is at least as similar to an anchor as the similarity threshold dictates.
  """
  similar = False
  for a in embedding_anchors:

    similar = tags_df.iloc[int(tag_num)][f"{ '_'.join(a.split(' ')) }_embed_dist"] < similarity_threshold
    if similar == True:
      return True
  return False

def text_to_tags(tag_num_string, similarity_threshold=9.45):
  """
  Convert the tag numbers given in the users dataset into text based on the values from the tags dataset.

  INPUT: Tags in number form

  NOTE: Can filter tags here by [(1) removing nonsense tags], (2) removing tags that are not
  within some similarity threshold to a target tag [phrase embeddings]

  IMPORTANT: Removing tags that are not within a threshold of similarity will reduce the computational
  load of permuting
  """
  tags_num_list = tag_num_string.split(" ")

  # Map the tag number to the text version of the tag if it is similar to one of the anchor tags.
  tags_text_list = [ tags_df.iloc[int(tag_num)]['clean'] if test_similarity(tag_num, similarity_threshold) else None for tag_num in tags_num_list ]

  # Remove some annoying, recurring tags
  tags_text_list = [ t if t not in ['nan', 'ict', ''] else None for t in tags_text_list ]

  # Filter out tags that were not similar to anchors
  tags_list = [str(t) for t in tags_text_list if t]

  global user_counter
  user_counter += 1

  if user_counter % 250 == 0:
    print("USER COUNTER:", user_counter)

  return ', '.join(tags_list)


In [ ]:
users_df['tag_list'] = users_df[0].apply( lambda x:  text_to_tags(tag_num_string=x, similarity_threshold=8.6) ) # Remove common decorative characters
users_df['tag_list'].head(20)

USER COUNTER: 250
USER COUNTER: 500
USER COUNTER: 750
USER COUNTER: 1000
USER COUNTER: 1250
USER COUNTER: 1500
USER COUNTER: 1750
USER COUNTER: 2000
USER COUNTER: 2250
USER COUNTER: 2500
USER COUNTER: 2750
USER COUNTER: 3000
USER COUNTER: 3250
USER COUNTER: 3500
USER COUNTER: 3750
USER COUNTER: 4000
USER COUNTER: 4250
USER COUNTER: 4500
USER COUNTER: 4750
USER COUNTER: 5000
USER COUNTER: 5250
USER COUNTER: 5500


0     academia, information saturated, learning styl...
1     multidisciplinary, numerical, cognitive lingui...
2     sensory system, second law of thermodynamics, ...
3     ontology learning, behavioral theories, emergi...
4        ontology learning, neuro dynamics, large scale
5     wiki studies, problema cientifico, e learning,...
6      ontology learning, predicting, reverse inference
7                           adaptive educational system
8     mid twentieth century, structural change, lear...
9     intelligent transportation systems, team scien...
10    digital researcher, readings, out there, speci...
11    scientific techniques, resources, out there, c...
12    thesaurus, interactive pathway tool, s a, reve...
13    cosmological theory, broad institute, gillespi...
14                                    ontology learning
15                             operational semantics, v
16    design principles, f, neto w, numerical method...
17                workshop test, sociotechnical 

In [ ]:
users_df = users_df.drop_duplicates()
users_df = users_df.dropna()
users_df['tag_list'] = users_df['tag_list'].apply(lambda x: str(x))

In [ ]:
users_df.describe()

,0,tag_list
count,5411,5411
unique,5411,5118
top,70 495 1631 2317 2526 2846 2931 3171 3297 3332...,
freq,1,119


In [ ]:
users_df = users_df.drop( users_df[users_df['tag_list'] == ''].index )

In [ ]:
users_df.describe()

,0,tag_list
count,5292,5292
unique,5292,5117
top,70 495 1631 2317 2526 2846 2931 3171 3297 3332...,ontology learning
freq,1,24


In [ ]:
users_df.head()

,0,tag_list
0,70 495 1631 2317 2526 2846 2931 3171 3297 3332...,"academia, information saturated, learning styl..."
1,38 493 942 1519 1843 1844 1896 2819 3391 5031 ...,"multidisciplinary, numerical, cognitive lingui..."
2,20 517 791 800 1329 1767 1984 2126 3009 4458 4...,"sensory system, second law of thermodynamics, ..."
3,12 706 709 721 755 756 776 892 895 2588 2991 4...,"ontology learning, behavioral theories, emergi..."
4,12 761 2678 4034 4035 5439 5494 5758 7798 9046...,"ontology learning, neuro dynamics, large scale"


In [ ]:
users_df.drop(0, axis=1, inplace=True)

users_df.to_csv('tags_by_user.csv', index=False)

In [ ]:
users_df = pd.read_csv('tags_by_user.csv')
users_df

,tag_list
0,"academia, information saturated, learning styl..."
1,"multidisciplinary, numerical, cognitive lingui..."
2,"sensory system, second law of thermodynamics, ..."
3,"ontology learning, behavioral theories, emergi..."
4,"ontology learning, neuro dynamics, large scale"
...,...
5287,"behavioral theories, organizational knowledge ..."
5288,"darwinian, system oriented"
5289,sensorimotor theory
5290,"instrumental conditioning, evolutionarily"


##### Make permutations

In [ ]:
# For each user, append a random number of rows of random permutations of their tags list.
import random
import numpy as np
permutations = []
for user in users_df['tag_list']: # For each user
  for i in range(random.choice(range(1, 6))): # Run a random number of times between 1 and 5
    random_perm = ", ".join( np.random.permutation( user.split(", ") ) )
    permutations.append(random_perm)
permutations_df = pd.DataFrame.from_dict( { 'tag_list': permutations } )

In [ ]:
permutations_df.describe()

,tag_list
count,15808
unique,13394
top,ontology learning
freq,80


In [ ]:
permutations_df = permutations_df.drop_duplicates()
permutations_df.describe()

,tag_list
count,13394
unique,13394
top,"qualitative methods, refactoring s, academia, ..."
freq,1


In [ ]:
permutations_df[13000:13050]

,tag_list
15334,"product recommendations, chemicals"
15336,"limnology, review, psych, artificial neural ne..."
15337,"retrospective study, disease propagation, limn..."
15338,"disease propagation, psych, artificial neural ..."
15339,"limnology, networks textbook, review, retrospe..."
15340,"information causality, in, function study, tea..."
15341,"aquatic, terrestrial, multi stage programming"
15343,"multi stage programming, aquatic, terrestrial"
15344,"multi stage programming, terrestrial, aquatic"
15345,"terrestrial, aquatic, multi stage programming"


In [ ]:
permute_df = users_df.append(permutations_df).sample(frac=1).drop_duplicates()
permute_df.describe()

,tag_list
count,17318
unique,17318
top,"selection algorithms, neuronal structures, bio..."
freq,1


In [ ]:
permute_df.head()

,tag_list
935,"selection algorithms, neuronal structures, bio..."
9379,"theory building, alter modernities, s a rig, l..."
6010,"political science, quantum information, life s..."
2947,"bionics, quantum simulation, brain power, beha..."
1964,"molecule detection, interactive pathway tool, ..."


In [ ]:
permute_df.to_csv('permuted_tags_by_user.csv', index=False)

#### Data augmentation

In [ ]:
ml_filter = users_df[users_df['tag_list'].str.contains("machine learning")].drop_duplicates()
ml_filter.describe()

In [ ]:
# IMPORTANT: Because n permutations grows really large, can generate N random permutations of the data for augmentation.

from math import factorial
def nPr(n, r):
    return int(factorial(n)/factorial(n-r))

n_machine_learning_related_tags, n_permutations = 0, 0
max_len = 0
for i in range(50):
  len_tags = len(ml_filter['tag_list'].iloc[i].split(','))
  max_len = len_tags if len_tags > max_len else max_len
  n_machine_learning_related_tags += len_tags
  n_permutations += nPr(len_tags, len_tags)

print(f"Total N tags = { n_machine_learning_related_tags }, Max len = {max_len}, # Permutations = { n_permutations }")

In [ ]:
permutations = list(itertools.permutations(ml_filter['tag_list'].iloc[9].split(',')))
[", ".join(i) for i in permutations]

In [ ]:
cs_filter = users_df[users_df['tag_list'].str.contains("computer science")]
cs_filter.describe()

In [ ]:
np.random.permutation(ml_filter['tag_list'].iloc[9].split(', '))

Sequences can be augmented in permutations so that each example creates nWords - 1 augmented examples. Split on ", "...